In [1]:
pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain youtube-transcript-api pytube


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = getpass.getpass("Langchain: ")
os.environ['OPENAI_API_KEY'] = getpass.getpass("Openai: ")

Langchain: ········
Openai: ········


In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

loader = WebBaseLoader("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/")
docs.extend(loader.load())

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
import uuid
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

chain = ({"doc": lambda x: x.page_content}
        | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
        | ChatOpenAI(model="gpt-3.5-turbo", max_retries = 0)
        | StrOutputParser()
        )
summaries = chain.batch(docs,{"max_concurrency" : 5})
summaries[0]

'The document discusses the concept of building autonomous agents powered by large language models (LLM). It covers key components of such agents including planning, memory, and tool use. The document provides examples of proof-of-concept demos like AutoGPT, GPT-Engineer, and BabyAGI, showcasing the potential of LLMs beyond generating text. Challenges such as finite context length, reliability of natural language interface, and long-term planning are also highlighted. Case studies like ChemCrow and Generative Agents Simulation demonstrate the application of LLM-powered agents in scientific discovery and behavior simulation. The document also references various studies and frameworks related to LLM-powered agents.'

In [9]:
from langchain.storage import InMemoryByteStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=OpenAIEmbeddings())

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Docs linked to summaries
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

# Add
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

/var/folders/kq/90ww7__n28d6svqwm1clq9g00000gn/T/ipykernel_5927/21713248.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(collection_name="summaries",


In [10]:
query = "Memory in agents"
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]

Document(metadata={'doc_id': '4b955642-5bf7-4d06-984c-fdbbc48cf54b'}, page_content='The document discusses the concept of building autonomous agents powered by large language models (LLM). It covers key components of such agents including planning, memory, and tool use. The document provides examples of proof-of-concept demos like AutoGPT, GPT-Engineer, and BabyAGI, showcasing the potential of LLMs beyond generating text. Challenges such as finite context length, reliability of natural language interface, and long-term planning are also highlighted. Case studies like ChemCrow and Generative Agents Simulation demonstrate the application of LLM-powered agents in scientific discovery and behavior simulation. The document also references various studies and frameworks related to LLM-powered agents.')